## Import Module

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os
import anndata
import omicverse as ov
import matplotlib as mpl
ov.plot_set(font_path='Arial')

# Enable auto-reload for development
%load_ext autoreload
%autoreload 2

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['text.usetex'] = False

sc.settings.verbosity = 1  
sc.settings.set_figure_params(dpi=80, figsize=(3, 3), facecolor="white")  
os.chdir('/home/data/gz0436/OneDrive_4_2025-3-8')
os.getcwd()

In [ ]:
adata = sc.read_h5ad('./semla_output/ctb_st_clean.h5ad')
adata

In [ ]:
# niche=pd.read_csv('./data/all_niche_subtype.csv',index_col=0)
niche=pd.read_csv('./data/all_niche_single.csv',index_col=0)
niche.head()

## Merge adata.obs and niche columns

In [ ]:
niche.shape

In [ ]:
# common_cells = set(niche['Cell_Barcode']).intersection(set(adata.obs_names))
# len(common_cells)
order_df = pd.DataFrame({'Cell_Barcode': adata.obs_names})
niche_sorted = order_df.merge(niche, on='Cell_Barcode', how='left')
niche_sorted.head()

In [ ]:
niche_sorted.tail()

In [ ]:
adata.obs_names

In [ ]:
pd.DataFrame(adata.obs_names==niche_sorted['Cell_Barcode']).value_counts()

In [ ]:
niche_sorted = niche_sorted.set_index('Cell_Barcode')

In [ ]:
niche_sorted = niche_sorted.apply(lambda x: x.astype('category')) 
niche_sorted.dtypes

In [ ]:
adata.obs[niche_sorted.columns]=niche_sorted
adata

## Check and Plot all clusters

In [ ]:
adata.obs

In [ ]:
kmeans_cols = adata.obs.filter(regex=r'^niches_\d+$').columns
kmeans_cols
adata.obs[kmeans_cols] = adata.obs[kmeans_cols].astype('category')
adata.obs['niches_10'].value_counts()

In [ ]:
pd.crosstab(adata.obs['first_type'],adata.obs['niches_10'])

In [ ]:
from plot_fun.spatial_utils import plot_resolution_spatial, get_cluster_palette
save_params = {'dpi': 300,'format': 'pdf', 'bbox_inches': 'tight','pad_inches': 0.5, 'metadata': {'pdf.fonttype': 42}}
fig=plot_resolution_spatial(adata, resolutions=['niches_20'])
# fig.savefig(f"./python_figures/First_type_rasterized.{save_params['format']}", **save_params)
plt.show()
plt.close()

In [ ]:
# custom_palette = {1: "#1036b5", 2: "#6f774b", 3: "#d10040", 4: "#84cd5d", 5: "#8E76DB", 
#                   6: "#71c8a5", 7: "#ffa26d", 8: "#924373", 9: "#a6513c", 10: "black", }
#                   11: "#72c7ff", 12: "#cabd00", 13: "#FF99FF", 14: "#00FFFF", 15: "#FF6600"}
palettes = [
    "#1036b5", "#6f774b", "#d10040", "#84cd5d", "#8E76DB", 
    "#71c8a5", "#ffa26d", "#924373", "#a6513c", "black",
    "#72c7ff", "#cabd00", "#FF99FF", "#00FFFF", "#FF6600",
    "#8A2BE2", "#20B2AA", "#FF6347", "#9370DB", "#32CD32"
]
all_clusters = adata.obs['niches_20'].astype('category').cat.categories
custom_palette = dict(zip(all_clusters, palettes[:len(all_clusters)]))

# all_clusters = adata.obs['niches_20'].astype('category').cat.categories
# custom_palette = dict(zip(all_clusters, sc.pl.palettes.default_102[:len(all_clusters)]))
# custom_palette = {cluster: sc.pl.palettes.default_102[i] for i, cluster in enumerate(all_clusters)}   
# plt.cm.tab20.colors
# custom_palette = dict(zip(all_clusters, plt.cm.tab20.colors))
                      
from plot_fun.plot_niches import plot_samples_cluster
plot_samples_cluster(
    adata,
    cluster_key='niches_20',
    legend_loc="right margin",
    palette=custom_palette,
    spot_size=50,
    alpha=0.9  
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Spatial All New.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
adata.write_h5ad('./semla_output/ctb_st_clean_niche.h5ad')

## Highlight specific clusters independently

In [ ]:
kmeans_cols = adata.obs.filter(regex=r'^niches_\d+$').columns
kmeans_cols

In [ ]:
from plot_fun.plot_niches import plot_cluster_highlights
plot_cluster_highlights(
    adata=adata,
    samples=['CTB1_BL',],
    resolutions=['niches_10', 'niches_15',],
    show=True,
    save=False,
    figsize_per_subplot = (10, 10),
    max_cols = 5,
    title_fontsize = 30,
    spot_size=50
)
plt.close()

In [ ]:
plot_cluster_highlights(
    adata=adata,
    samples=['CTB1_BL',],
    resolutions=['niches_20',],
    show=True,
    save=False,
    figsize_per_subplot = (10, 10),
    max_cols = 5,
    title_fontsize = 30,
    spot_size=50
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Split-CTB1_BL.pdf", dpi=300, bbox_inches="tight")
plt.close()

In [ ]:
plot_cluster_highlights(
    adata=adata,
    samples=['CTB7_BL',],
    resolutions=['niches_10',],
    show=True,
    save=False,
    figsize_per_subplot = (10, 10),
    max_cols = 5,
    title_fontsize = 30,
    spot_size=50
)
plt.close()

## Plot Niche Proportion

In [ ]:
adata

In [ ]:
crosstab = pd.crosstab(
    adata[adata.obs['sample']=="CTB1_BL",].obs['first_type'], 
    adata[adata.obs['sample']=="CTB1_BL",].obs['niches_10'], 
    normalize='index'
)
crosstab

In [ ]:
proportion_df = crosstab.reset_index().melt(id_vars='first_type', var_name='niches', value_name='Proportion')
proportion_df

In [ ]:
# %reload_ext autoreload
# %autoreload 2

from plot_fun.plot_niches import plot_niche_proportion
palettes = [
    "#1036b5", "#6f774b", "#d10040", "#84cd5d", "#8E76DB", 
    "#71c8a5", "#ffa26d", "#924373", "#a6513c", "black",
    "#72c7ff", "#cabd00", "#FF99FF", "#00FFFF", "#FF6600",
    "#8A2BE2", "#20B2AA", "#FF6347", "#9370DB", "#32CD32"
]
all_clusters = adata.obs['niches_20'].astype('category').cat.categories
custom_palette = dict(zip(all_clusters, palettes[:len(all_clusters)]))
# custom_palette = {cluster: sc.pl.palettes.default_102[i] for i, cluster in enumerate(all_clusters)}   
# plt.cm.tab20.colors
                      
fig, ax = plot_niche_proportion(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    sample='CTB7_BL',
    color_dict=custom_palette ,
    figsize=(16, 7), 
    size_range=(30, 500),
    fontsize=12,
    legend_columns=1, 
    min_proportion=0.01  
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB7_BL.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax = plot_niche_proportion(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    sample='CTB3_BL',
    color_dict=custom_palette ,
    figsize=(16, 7), 
    size_range=(30, 500),
    fontsize=12,
    legend_columns=1, 
    min_proportion=0.01  
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB3_BL.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
all_clusters = adata.obs['niches_20'].astype('category').cat.categories
custom_palette = dict(zip(all_clusters, palettes[:len(all_clusters)]))
fig, ax = plot_niche_proportion(adata, celltype_col='first_type', niche_col='niches_20', sample='CTB1_BL', color_dict=custom_palette,)
plt.show()
plt.close()

## Plot Niche Proportion With Categories

In [ ]:
celltype_to_category = {
    # Keratinocytes
    'Basal KC': 'Keratinocyte', 'Inflammatory KC': 'Keratinocyte', 'Proliferating KC': 'Keratinocyte',
    'Spinous KC': 'Keratinocyte', 'Granular KC': 'Keratinocyte',
    # Others
    'Hair follicle': 'Others', 'Sweet Gland': 'Others', 'Schwann Cell': 'Others', 'Melanocyte': 'Others',
    # Fibroblasts
    'CCL19+ FB': 'Fibroblast', 'APCDD1+ FB': 'Fibroblast', 'PI16+ FB': 'Fibroblast',
    'POSTN+ FB': 'Fibroblast', 'TNN+ FB': 'Fibroblast', 'RAMP1+ FB': 'Fibroblast',
    # Endothelial cells
    'Arterial EC': 'Endothelial', 'Capillary EC': 'Endothelial', 'Venular EC1': 'Endothelial',
    'Venular EC2': 'Endothelial', 'Lymphatic Endothelial': 'Endothelial', 'Smooth muscle': 'Endothelial',
    # Myeloid cells
    'Mast cell': 'Myeloid', 'Langerhans': 'Myeloid', 'cDC1': 'Myeloid', 'cDC2A': 'Myeloid',
    'cDC2B': 'Myeloid', 'pDC': 'Myeloid', 'M1-like Mac': 'Myeloid', 'M2-like Mac': 'Myeloid',
    'SPP1 Mac': 'Myeloid', 'TREM2 Mac': 'Myeloid',
     # T cells
    'CD8T': 'T cell', 'Treg': 'T cell', 'CD4 Naïve': 'T cell', 'Th17': 'T cell',
    'CD8T_NK': 'T cell', 'Th17_Treg': 'T cell', 'NK': 'T cell',
    # B cells
    'naïve B': 'B cell', 'activated B': 'B cell', 'memory B': 'B cell', 'Plasma': 'B cell'
}

In [ ]:
from plot_fun.plot_niches import plot_niche_proportion_with_categories
all_clusters = adata.obs['niches_10'].astype('category').cat.categories
custom_palette = dict(zip(all_clusters, palettes[:len(all_clusters)]))
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_10',
    celltype_to_category=celltype_to_category,
    sample='CTB1_BL',
    color_dict=custom_palette,
    figsize=(16, 7), 
    size_range=(30, 500),
    fontsize=12,
    legend_columns=1, 
    min_proportion=0.01,
    category_spacing=0.4
)
plt.show()
plt.close()

In [ ]:
all_clusters = adata.obs['niches_15'].astype('category').cat.categories
custom_palette = dict(zip(all_clusters, palettes[:len(all_clusters)]))
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_15',
    celltype_to_category=celltype_to_category,
    sample='CTB1_BL',
    color_dict=custom_palette,
    figsize=(16, 8), 
)
plt.show()
plt.close()

In [ ]:
all_clusters = adata.obs['niches_20'].astype('category').cat.categories
custom_palette = dict(zip(all_clusters, palettes[:len(all_clusters)]))
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    celltype_to_category=celltype_to_category,
    sample='CTB1_BL',
    color_dict=custom_palette,
    figsize=(16, 8), 
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB1_BL.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
all_clusters = adata.obs['niches_20'].astype('category').cat.categories
custom_palette = dict(zip(all_clusters, palettes[:len(all_clusters)]))
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    celltype_to_category=celltype_to_category,
    sample='CTB1_T',
    color_dict=custom_palette,
    figsize=(16, 8), 
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB1_T.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    celltype_to_category=celltype_to_category,
    sample='CTB3_BL',
    color_dict=custom_palette,
    figsize=(16, 8), 
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB3_BL.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    celltype_to_category=celltype_to_category,
    sample='CTB3_T',
    color_dict=custom_palette,
    figsize=(16, 8), 
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB3_T.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    celltype_to_category=celltype_to_category,
    sample='CTB7_BL',
    color_dict=custom_palette,
    figsize=(16, 8), 
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB7_BL.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax = plot_niche_proportion_with_categories(
    adata,
    celltype_col='first_type',
    niche_col='niches_20',
    celltype_to_category=celltype_to_category,
    sample='CTB7_T',
    color_dict=custom_palette,
    figsize=(16, 8), 
)
plt.tight_layout()
plt.savefig("./python_figures/Cell Niches Composition-CTB7_T.pdf", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

## SPP1_Mac_LGC_Niche High exp genes

In [ ]:
adata_1BL=adata[adata.obs['sample']=='CTB1_BL'].copy()
adata_1BL

In [ ]:
sc.tl.rank_genes_groups(adata_1BL, 'niches_10', method="wilcoxon")
adata_1BL

In [ ]:
fig, ax = plt.subplots(figsize=(15,6), dpi=100)
sc.pl.rank_genes_groups(adata_1BL, n_genes=15, sharey=False, ncols = 5, ax =ax, show = False)
ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')
plt.show()
plt.close()

In [ ]:
df_genes = pd.DataFrame(adata_1BL.uns['rank_genes_groups']['names']).head(50)
df_genes.columns.name = "Niche"  
df_genes.index.name = "Gene_Rank"  
df_genes.head(50)

In [ ]:
from plot_fun.spatial_utils import plot_dual_features
fig = plot_dual_features(adata, 'FBP1', 'LDHA', normalize='sample', title=None)
# fig.savefig(f'./python_figures/SPP1_Mac_TREM2_Mac_Spatial_Dis.pdf', dpi=300, bbox_inches='tight', transparent=False) 
plt.show()
plt.close()

In [ ]:
adata_1BL.obs['ref_group'] = adata_1BL.obs['niches_10'].astype(str)
adata_1BL.obs['ref_group'] = adata_1BL.obs['ref_group'].replace(
    {'M1-like Mac': "ref", 'M2-like Mac': "ref", 'TREM2 Mac': "ref"}
)
adata_1BL.obs['ref_group'].unique()